In [2]:
from platform import python_version

print('python version of this jupyter notebook:', python_version())

python version of this jupyter notebook: 3.7.6


In [3]:
import pandas as pd
from tqdm.notebook import tqdm
from collections import defaultdict
import numpy as np

In [4]:
movies = pd.read_csv('./archive/IMDb movies.csv')

/Users/markzuckerberg/anaconda3/envs/wintertime/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
print('the length of the dataset is:', len(movies))

the length of the dataset is: 85855


In [6]:
# use 70000 of the dataset as the training set and the rest as the testing set
train_set = movies[:70000]
test_set = movies[70000:]

In [7]:
train_set.head()

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,...,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
0,tt0000009,Miss Jerry,Miss Jerry,1894,1894-10-09,Romance,45,USA,None,Alexander Black,...,"Blanche Bayliss, William Courtenay, Chauncey D...",The adventures of a female reporter in the 1890s.,5.9,154,NaN,NaN,NaN,NaN,1.0,2.0
1,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,1906-12-26,"Biography, Crime, Drama",70,Australia,None,Charles Tait,...,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,6.1,589,$ 2250,NaN,NaN,NaN,7.0,7.0
2,tt0001892,Den sorte drøm,Den sorte drøm,1911,1911-08-19,Drama,53,"Germany, Denmark",NaN,Urban Gad,...,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...,5.8,188,NaN,NaN,NaN,NaN,5.0,2.0
3,tt0002101,Cleopatra,Cleopatra,1912,1912-11-13,"Drama, History",100,USA,English,Charles L. Gaskill,...,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",The fabled queen of Egypt's affair with Roman ...,5.2,446,$ 45000,NaN,NaN,NaN,25.0,3.0
4,tt0002130,L'Inferno,L'Inferno,1911,1911-03-06,"Adventure, Drama, Fantasy",68,Italy,Italian,"Francesco Bertolini, Adolfo Padovan",...,"Salvatore Papa, Arturo Pirovano, Giuseppe de L...",Loosely adapted from Dante's Divine Comedy and...,7.0,2237,NaN,NaN,NaN,NaN,31.0,14.0


In [8]:
actor_movies =  {}
actor_count = {}
vote_add = 0
for index, row in tqdm(train_set.iterrows()):
    if type(row['actors']) != str:
        continue
    actors = row['actors'].split(',')
    for a in actors:
        if a not in actor_movies:
            actor_movies[a] = {row['imdb_title_id']}
            actor_count[a] = {0}
        else:
            actor_movies[a].add(row['imdb_title_id'])
            actor_count[a] 
    vote_add += row['avg_vote']
    
vote_mean = vote_add/len(train_set)

In [9]:
# count the number of movies an actor plays
actor_count = defaultdict(int)
actor_rate = defaultdict(int)

for index, row in tqdm(train_set.iterrows()):
    if type(row['actors']) != str:
        continue
    actors = row['actors'].split(',')
    for a in actors:
        a = a.replace(" ", "")
        actor_count[a] += 1
        actor_rate[a] += row['avg_vote']
        if a == "Angibell": print(row['title'], row['avg_vote'])

Lejos de Casa pelicula Venezolana 9.8



In [10]:
for a in actor_rate:
    actor_rate[a] = actor_rate[a]/actor_count[a]
    
actor_rate = sorted(actor_rate.items(), key= lambda x: x[1], reverse=True)
actor_count = sorted(actor_count.items(), key= lambda x: x[1], reverse=True)

In [11]:
genres = []
genreSet = set()
for index, row in train_set.iterrows():
    genres.append([])
    for t in row['genre'].split(', '):
        genreSet.add(t)
        genres[-1].append(t)
numGenre = len(genreSet)
genreToIndex = dict()
for i, g in enumerate(list(genreSet)):
    genreToIndex[g] = i
for i, genre in enumerate(genres):
    t = [0] * numGenre
    for g in genre:
        t[genreToIndex[g]] = 1
    genres[i] = t
    
print(genres)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [12]:
countries = []
tmp = set()
frequency = defaultdict(int)
for index, row in train_set.iterrows():
    countries.append([])
#     print(index, row['country'], row['title'])
    if type(row['country']) != str: continue
    for t in row['country'].split(', '):
        tmp.add(t)
        frequency[t] += 1
        countries[-1].append(t)
numCountries = len(tmp)
frequency = sorted(frequency.keys(), key=lambda x: frequency[x], reverse=True)[:10]
print(frequency)
countryToIndex = dict()
for i, c in enumerate(frequency):
    countryToIndex[c] = i
for i, country in enumerate(countries):
    t = [0] * 10
    for c in country:
        if c in frequency:
            t[countryToIndex[c]] = 1
    countries[i] = t
    
# print(countries)

['USA', 'France', 'UK', 'Italy', 'India', 'Japan', 'Germany', 'Canada', 'Spain', 'Hong Kong']


In [13]:
print(countries[2000])
for index, row in train_set.iterrows():
    if index == 2000: 
        print(row['country'])
        break

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
USA


In [14]:
directorAndHisGenre = defaultdict(list)
m = 0

for index, row in train_set.iterrows():
    if type(row['director']) != str: continue 
    for t in row['director'].split(', '):
        if len(directorAndHisGenre[t]) == 0: directorAndHisGenre[t] = [0 for _ in range(2 * numGenre)]
        for g in row['genre'].split(', '):
            directorAndHisGenre[t][genreToIndex[g]] += 0.1
            directorAndHisGenre[t][genreToIndex[g]+numGenre] += row['avg_vote'] / 20
            m = max(m, directorAndHisGenre[t][genreToIndex[g]+numGenre])
        
print(directorAndHisGenre)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [15]:
genreStat = [sum([n[i] for n in genres]) / len(genres[0]) for i in range(len(genres[0]))]
print(genreStat)
genreToIndex
print(sum(genreStat))

[74.68, 346.6, 60.52, 0.04, 76.56, 0.08, 81.32, 423.6, 259.24, 1544.84, 126.28, 75.16, 34.48, 0.08, 383.48, 966.12, 0.04, 26.52, 293.48, 126.76, 58.6, 119.24, 166.88, 63.76, 481.08]
5789.440000000001


In [16]:
actorAndHisGenre = defaultdict(list)
m = 0

for index, row in train_set.iterrows():
    if type(row['actors']) != str: continue 
    for t in row['actors'].split(', '):
        tmpName = t.replace(' ', '')
        if len(actorAndHisGenre[tmpName]) == 0: actorAndHisGenre[tmpName] = [0 for _ in range(2 * numGenre)]
        for g in row['genre'].split(', '):
            actorAndHisGenre[tmpName][genreToIndex[g]] += 0.1
            actorAndHisGenre[tmpName][genreToIndex[g]+numGenre] += row['avg_vote'] / 20
#             m = max(m, directorAndHisGenre[t][genreToIndex[g]+numGenre])
        
print(actorAndHisGenre.keys())

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [17]:
print(list(actorAndHisGenre.keys())[:10])

['BlancheBayliss', 'WilliamCourtenay', 'ChaunceyDepew', 'ElizabethTait', 'JohnTait', 'NormanCampbell', 'BellaCola', 'WillCoyne', 'SamCrewes', 'JackEnnis']


In [18]:
years = []
for index, row in tqdm(train_set.iterrows()):
    t = [0] * 10
    year = row['year']
    i = 9
    if int(year) <= 1930:
        i = 1
    else: 
        i = int((int(year) - 1921)/10)
    t[i] = 1
    years.append(t)

In [19]:
X = []
for index, row in tqdm(train_set.iterrows()):
    directorFeature = directorAndHisGenre[row['director']]
#     print(len(years[index]), len(directorFeature), len(genres[index]), len(countries[index]))
    X.append(years[index] + (directorFeature if len(directorFeature) > 0 else [0] * (2 * numGenre)) + genres[index] + countries[index])
    actorFeature = [0 for _ in range(2 * numGenre)]
    if type(row['actors']) == str: 
        for actor in row['actors'].split(', '):
            for i, x in enumerate(actorAndHisGenre[actor.replace(' ', '')]):
                actorFeature[i] += x
    X[-1] = X[-1] + actorFeature

In [20]:
y = []
for index, row in tqdm(train_set.iterrows()):
    y.append(row['avg_vote'])

In [21]:
X = np.array(X)
y = np.array(y)



#### build test set

In [22]:
directorAndHisGenre = defaultdict(list)
m = 0

for index, row in test_set.iterrows():
    if type(row['director']) != str: continue 
    for t in row['director'].split(', '):
        if len(directorAndHisGenre[t]) == 0: directorAndHisGenre[t] = [0 for _ in range(2 * numGenre)]
        for g in row['genre'].split(', '):
            directorAndHisGenre[t][genreToIndex[g]] += 0.1
            directorAndHisGenre[t][genreToIndex[g]+numGenre] += row['avg_vote'] / 20
            m = max(m, directorAndHisGenre[t][genreToIndex[g]+numGenre])

In [23]:
countries = []
tmp = set()
frequency = defaultdict(int)
for index, row in test_set.iterrows():
    countries.append([])
#     print(index, row['country'], row['title'])
    if type(row['country']) != str: continue
    for t in row['country'].split(', '):
        tmp.add(t)
        frequency[t] += 1
        countries[-1].append(t)
numCountries = len(tmp)
frequency = sorted(frequency.keys(), key=lambda x: frequency[x], reverse=True)[:10]
print(frequency)
countryToIndex = dict()
for i, c in enumerate(frequency):
    countryToIndex[c] = i
for i, country in enumerate(countries):
    t = [0] * 10
    for c in country:
        if c in frequency:
            t[countryToIndex[c]] = 1
    countries[i] = t

['USA', 'India', 'France', 'UK', 'Germany', 'Canada', 'Japan', 'Italy', 'Belgium', 'Turkey']


In [24]:
genres = []
genreSet = set()
for index, row in test_set.iterrows():
    genres.append([])
    for t in row['genre'].split(', '):
        genreSet.add(t)
        genres[-1].append(t)
numGenre = len(genreSet)
genreToIndex = dict()
for i, g in enumerate(list(genreSet)):
    genreToIndex[g] = i
for i, genre in enumerate(genres):
    t = [0] * numGenre
    for g in genre:
        t[genreToIndex[g]] = 1
    genres[i] = t

In [25]:
actorAndHisGenre = defaultdict(list)
m = 0

for index, row in test_set.iterrows():
    if type(row['actors']) != str: continue 
    for t in row['actors'].split(', '):
        tmpName = t.replace(' ', '')
        if len(actorAndHisGenre[tmpName]) == 0: actorAndHisGenre[tmpName] = [0 for _ in range(2 * numGenre)]
        for g in row['genre'].split(', '):
            actorAndHisGenre[tmpName][genreToIndex[g]] += 0.1
            actorAndHisGenre[tmpName][genreToIndex[g]+numGenre] += row['avg_vote'] / 20

In [35]:
years = []
for index, row in tqdm(test_set.iterrows()):
    t = [0] * 10
    year = row['year']
    if year.isdecimal():
        i = 9
        if int(year) <= 1930:
            i = 1
        else: 
            i = int((int(year) - 1921)/10)
        t[i] = 1
    years.append(t)


In [27]:
len(test_set)

15855

In [28]:
print(len(years), len(directorAndHisGenre), len(genres), len(countries))

0 12640 15855 15855


In [37]:
X = []

for index, row in tqdm(test_set.iterrows()):
    directorFeature = directorAndHisGenre[row['director']]

    index = index - 70000

    #print(len(years[index]), len(directorFeature), len(genres[index]), len(countries[index])) 
    X.append(years[index] + (directorFeature if len(directorFeature) > 0 else [0] * (2 * numGenre)) + genres[index] + countries[index])
    actorFeature = [0 for _ in range(2 * numGenre)]
    if type(row['actors']) == str: 
        for actor in row['actors'].split(', '):
            for i, x in enumerate(actorAndHisGenre[actor.replace(' ', '')]):
                actorFeature[i] += x
    X[-1] = X[-1] + actorFeature

In [38]:
y = []
for index, row in tqdm(test_set.iterrows()):
    y.append(row['avg_vote'])

In [39]:
test_X = np.array(X)
test_y = np.array(y)

In [40]:
len(X[0])

136

In [45]:
len(test_X[0])

136